# Sentiment Classification Using DistilBERT 

# What is `DistilBERT`

BERT is designed to pretrain deep bidirectional representations from
unlabeled text by jointly conditioning on both
left and right context in all layers. As a result, the pre-trained BERT model can be finetuned with just one additional output layer
to create state-of-the-art models for a wide
range of tasks, such as question answering and
language inference, without substantial taskspecific architecture modifications.

DistilBERT is a small, fast, cheap and light Transformer model trained by distilling Bert base. It has 40% less parameters than bert-base-uncased, runs 60% faster while preserving over 95% of Bert’s performances as measured on the GLUE language understanding benchmark.

# What is `ktrain`

ktrain is a library to help build, train, debug, and deploy neural networks in the deep learning software framework, Keras.

# Notebook Setup

In [ ]:
!pip install ktrain

     |████████████████████████████████| 25.2 MB 3.2 MB/s 
     |████████████████████████████████| 421.8 MB 22 kB/s 
     |████████████████████████████████| 6.7 MB 40.2 MB/s 
     |████████████████████████████████| 981 kB 7.3 MB/s 
     |████████████████████████████████| 241 kB 39.9 MB/s 
     |████████████████████████████████| 468 kB 48.3 MB/s 
     |████████████████████████████████| 448 kB 40.8 MB/s 
     |████████████████████████████████| 3.8 MB 25.2 MB/s 
     |████████████████████████████████| 50 kB 4.7 MB/s 
     |████████████████████████████████| 496 kB 46.1 MB/s 
  Created wheel for ktrain: filename=ktrain-0.18.4-py3-none-any.whl size=25253171 sha256=214c942c6dfe040841d2fc9e35516423ed58d5e749759640197e21e203f9186f
  Stored in directory: /root/.cache/pip/wheels/44/98/d9/a96450b37bdacc871c00a6da8ec6cecf9a6d166b299d7ee3b5
  Created wheel for keras-bert: filename=keras_bert-0.85.0-py3-none-any.whl size=34302 sha256=7e5471e71ae8341a062584d37b8425b27c6993f1839b47e7bb46b0d189f2ad6e
  S

In [ ]:
!git clone https://github.com/sarthak-sriw/IMDB-Movie-Reviews-Large-Dataset-50k.git

Cloning into 'IMDB-Movie-Reviews-Large-Dataset-50k'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 10 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (10/10), done.


In [ ]:
# /content/IMDB-Movie-Reviews-Large-Dataset-50k

In [ ]:
import pandas as pd
import numpy as np
import ktrain
from ktrain import text
import tensorflow as tf

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [ ]:
data_test = pd.read_excel('./IMDB-Movie-Reviews-Large-Dataset-50k/train.xlsx', dtype= str)
data_train = pd.read_excel('./IMDB-Movie-Reviews-Large-Dataset-50k/test.xlsx', dtype = str)

In [ ]:
data_train.sample(5)

,Reviews,Sentiment
18490,"This movie is bizarre. Better put, it's ""freak...",neg
20313,Hobgoblins... what a concept. Rick Sloan was a...,neg
7200,"Wonderful film, one of the best horror films o...",pos
10818,"I may be getting ahead of myself here, but alt...",pos
12539,This film is wonderful in every way that moder...,pos


In [ ]:
text.print_text_classifiers()

fasttext: a fastText-like model [http://arxiv.org/pdf/1607.01759.pdf]
logreg: logistic regression using a trainable Embedding layer
nbsvm: NBSVM model [http://www.aclweb.org/anthology/P12-2018]
bigru: Bidirectional GRU with pretrained fasttext word vectors [https://fasttext.cc/docs/en/crawl-vectors.html]
standard_gru: simple 2-layer GRU with randomly initialized embeddings
bert: Bidirectional Encoder Representations from Transformers (BERT) [https://arxiv.org/abs/1810.04805]
distilbert: distilled, smaller, and faster BERT from Hugging Face [https://arxiv.org/abs/1910.01108]


In [ ]:
(train, val, preproc) = text.texts_from_df(train_df=data_train, text_column='Reviews', label_columns='Sentiment',
                   val_df = data_test,
                   maxlen = 400,
                   preprocess_mode = 'distilbert')

preprocessing train...
language: en
train sequence lengths:
	mean : 234
	95percentile : 598
	99percentile : 913


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 234
	95percentile : 598
	99percentile : 913


In [ ]:
model = text.text_classifier(name = 'distilbert', train_data = train, preproc=preproc)

Is Multi-Label? False
maxlen is 400



done.


In [ ]:
learner = ktrain.get_learner(model = model,
                             train_data = train,
                             val_data = val,
                             batch_size = 6)

In [ ]:
learner.fit_onecycle(lr = 2e-5, epochs=2)



begin training using onecycle policy with max lr of 2e-05...
Train for 4167 steps, validate for 782 steps
Epoch 1/2
4167/4167 [==============================] - 1016s 244ms/step - loss: 0.2938 - accuracy: 0.8736 - val_loss: 0.1630 - val_accuracy: 0.9428
Epoch 2/2
4167/4167 [==============================] - 997s 239ms/step - loss: 0.1592 - accuracy: 0.9400 - val_loss: 0.0637 - val_accuracy: 0.9828


In [ ]:
predictor = ktrain.get_predictor(learner.model, preproc)

In [ ]:
predictor.save('./')

In [ ]:
data = ['this movie was really bad. acting was also bad. I will not watch again',
        'the movie was really great. I will see it again', 'another great movie. must watch to everyone']

In [ ]:
predictor.predict(data)

['neg', 'pos', 'pos']

In [ ]:
predictor.get_classes()

['neg', 'pos']

In [ ]:
predictor.predict(data, return_proba=True)

array([[0.994142  , 0.00585801],
       [0.00469813, 0.99530184],
       [0.00349588, 0.9965042 ]], dtype=float32)